In [ ]:
import os
user = os.getenv('USER')
import sys
sys.path.insert(0, f'/Home/{user}/py/stereoid')

In [ ]:
from datetime import datetime
import glob
from multiprocessing import Pool

import numpy as np
import matplotlib.pyplot as plt
import pyproj

import drama.utils as drtls
from drama.io import cfg as cfg
from drama.geo import SingleSwathBistatic

import stereoid.sar_performance as strsarperf
import stereoid.oceans.tools.observation_tools as obs_tools
from stereoid.sea_ice import FwdModel, SceneGenerator, RetrievalModel

from harmony23lib import *

In [ ]:
# Run ID
mode = "IWS"
runid='2021_1_seaice'

# stereoid path
path=f'/data1/{user}/stereoid'
pardir=path + '/PAR/'
parfile=pardir + 'Hrmny_' + '2021_1_seaice' + '.cfg'
cfgdata = cfg.ConfigFile(drtls.get_par_file(parfile))
main_dir=path + ''

# input neXtSIM files
model_dir=path + '/nextsim/'

# radar model
#fstr_dual = strsarperf.sarperf_files(main_dir, rx_dual_name, mode=mode, runid=runid) # at the moment IW only I think
fstr_ati = strsarperf.sarperf_files(main_dir, 'tud_2020_tripple_ati', mode=mode, runid=runid)
fstr_s1 = strsarperf.sarperf_files(main_dir, 'sentinel', mode=mode, runid=runid, is_bistatic=False)

# some additional settings
x_res=cfgdata.sar.gr_res # is readily set 
y_res=cfgdata.sar.gr_res # make it equal to the monostatic ground-range resolution
t_res=cfgdata.orbit.timestep # along-track time resolution of generated swath
sp=[complex(1.6,0.07), 0.0030, 0.015, 0.04] 
ip=[complex(3.65,0.38), 0.0030, 0.015] 
pol='hh+hv'
prod_res = np.sqrt(x_res * y_res)
b_ati=10
t_orb=12/175

inc_m_deg=31.1
inc_m = np.deg2rad(inc_m_deg)
along_track_separation=375e3 # should be consistent with the PAR file
obs_geo = obs_tools.build_geometry(parfile, inc_m, dau=along_track_separation)

fwdm = FwdModel(parfile,sp,ip)# initialize forwaes model
sgm = SceneGenerator(fwdm,x_res,y_res,n_orbs=2) # initialize scene generator
proj = pyproj.proj.Proj('+proj=stere +lat_0=90 +lat_ts=75')
n_days = 1
n_orbits_per_day = 3
n_orbits = n_days * n_orbits_per_day
# output folder
outpath=f'/data1/{user}/stereoid/RESULTS/SeaIce/SeaIceDrift'

In [ ]:
def process_input_dir(input_dir_mask):
    """
    Process all files from one directory.

    Inputs:
        input_dir_mask : str, mask of files in one dir, e.g. '/data/experiment01/field_2020*npz'

    Outputs are NPZ files. They contain velocities and coordinates and are stored in <outpath>
    
    """
    input_dir, mask = os.path.split(input_dir_mask)
    root_input_dir, exp_dir = os.path.split(input_dir)
    files = [os.path.basename(f) for f in sorted(glob.glob(f'{input_dir_mask}'))]
    dates = [f.split('_')[1].split('Z')[0] for f in files]
    dates = [datetime(int(d[0:4]), int(d[4:6]), int(d[6:8]), int(d[9:11])) for d in dates]
    t_ns = np.array([(d - dates[0]).total_seconds()/24/60/60 for d in dates])

    print(len(files), files[0], files[-1])
    swb = SingleSwathBistatic(par_file=parfile, dau=along_track_separation, n_orbits=n_orbits)
    for n_orbit in range(n_orbits):
        t = n_orbit * t_orb
        I = np.argmin(np.absolute(t-t_ns))
        filename = files[I]
        print(input_dir, n_orbit, filename, t, I)
        try:
            swth_bst = next(swb)
            print(n_orbit, 'OK', swth_bst.master_swath.Northing.shape)
        except:
            print(n_orbit, 'Error creating orbit geometry')
            continue
        x_sz, y_sz, nor_sz = create_swath_grids(swth_bst, proj, t_res, y_res)
        x, y, v_e, v_n, c, t = read_nextsim_data(f'{input_dir}/{filename}', proj)
        v_e_sz, v_n_sz, landmask, icemask = interpolate_nextsim_on_swath(x, y, v_e, v_n, c, t, x_sz, y_sz)
        u_int, v_int = compute_nextsim_uv(v_e_sz, v_n_sz, landmask, nor_sz)
        dopp, s_dopp, r_dopp_r0 = get_doppler(sgm, obs_geo, u_int, v_int, x_res, pol, inc_m, fstr_s1, fstr_ati, prod_res, b_ati)
        r_dopp_d0 = remove_texture_noise(r_dopp_r0, s_dopp)

        # get indices of large land free, icy chuncks
        ch_starts, ch_stops, ch_sizes = get_chunks(landmask, icemask)
        # process chunk-by-chunk
        for ch_i, (ch_start, ch_stop, ch_size) in enumerate(zip(ch_starts, ch_stops, ch_sizes)):
            r_dopp_r = r_dopp_r0[ch_start:ch_stop]
            r_dopp_a = np.zeros_like(r_dopp_r)
            r_dopp_m = np.zeros_like(r_dopp_r)
            for i in range(3):
                r_dopp_m[:,:,i] = denoise_mk(dopp[ch_start:ch_stop,:,i], r_dopp_d0[ch_start:ch_stop,:,i])
                r_dopp_a[:,:,i] = apply_anisotropic_diffusion(r_dopp_m[:,:,i], kappa=5)

            retrievalm = RetrievalModel(obs_geo.concordia, obs_geo.discordia, parfile)
            uv_r = retrievalm.sea_ice_drift(np.array(r_dopp_r)) # from raw Doppler data
            uv_m = retrievalm.sea_ice_drift(np.array(r_dopp_m)) # from Doppler data filtered with texture denoising and MK filrer
            uv_a = retrievalm.sea_ice_drift(np.array(r_dopp_a)) # from Doppler data filtered with texture denoising, MK and AD filter

            ofile = f'{outpath}/pass_{exp_dir}_{filename[6:-4]}_{n_orbit:02}_{ch_i:02}.npz'
            print(ofile)
            np.savez(ofile, 
                     uv_i=np.dstack([u_int[ch_start:ch_stop], v_int[ch_start:ch_stop]]), # interpolated input
                     uv_r=uv_r,
                     uv_m=uv_m,
                     uv_a=uv_a,
                     x = x_sz[ch_start:ch_stop],
                     y = y_sz[ch_start:ch_stop],
                     n = nor_sz[ch_start:ch_stop],
                    )

def compute_defor(ifile, stp=2):
    """ Compute deformation from using MK and AK methods.
    
    Inputs:
        ifile : str, name of input file
        stp :, int, zoomout factor
    Outputs are stored in NPZ files and contain 3 types of deformation fields:
        ???i - from original velocities
        ???m - using MK method
        ???a using AK method
        
        """
    ofile = ifile.replace('.npz', '_defor.npz')
    ds = np.load(ifile)

    ui = ds['uv_i'][:,:,0]
    vi = ds['uv_i'][:,:,1]
    divi, shei, toti = get_deformation(ui[::stp, ::stp], vi[::stp, ::stp])

    um = ds['uv_m'][:,:,0]
    vm = ds['uv_m'][:,:,1]
    divm, shem, totm = get_deformaion_from_uv_mk2(um, vm, stp=stp)

    ua = ds['uv_a'][:,:,0]
    va = ds['uv_a'][:,:,1]
    uz = multi_look(ua, stp)
    vz = multi_look(va, stp)
    uc, vc = clustering_filter(uz, vz, n_clusters=50, med_filt_size=5, xy_interpolation=True)
    divc, shec, totc = get_deformation(uc, vc)
    np.savez(ofile, divi=divi, shei=shei, toti=toti, divm=divm, shem=shem, totm=totm, divc=divc, shec=shec, totc=totc, stp=stp)

def make_quiclook(ifile1, uv_names, df_names):
    """
    Plot U/V and deformation on a PNG file. 
    
    Inputs:
        ifile1 : str, input file
        uv_names : list of str, name of U/V variables in input file to visualise
        df_names : list of str, name of deformation variables in input file to visualise
    Outputs are saved as PNG files with the same name as input file but with PNG extension.
    
    """
    ifile2 = ifile1.replace('.npz', '_defor.npz')
    ofile = ifile2.replace('_defor.npz', '_ql.png')

    ds1 = np.load(ifile1)
    ds2 = np.load(ifile2)

    n_maps = len(uv_names + df_names)
    fig, axs = plt.subplots(n_maps, 1, figsize=(10,2*n_maps))
    i = 0

    for uv_name in uv_names:
        if uv_name in ds1.files:
            u = ds1[uv_name][:,:,0]
            axs[i].imshow(u.T, clim=[-0.1, 0.1], cmap='bwr')
            axs[i].set_title(uv_name)
            i += 1

    for df_name in df_names:
        if df_name in ds2.files:
            d = ds2[df_name]
            axs[i].imshow(d.T, clim=[0, 0.02], cmap='plasma_r')
            axs[i].set_title(df_name)
            i += 1

    plt.savefig(ofile, dpi=100, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
# compute velocities in paralel
input_dir_masks = [
    f'/data1/{user}/stereoid/nextsim/original_data/field_20190101*00Z.npz',
    f'/data1/{user}/stereoid/nextsim/newer_exp/field_20070105*00Z.npz',
    f'/data1/{user}/stereoid/nextsim/sa05free_2019/field_20190101*00Z.npz',
]

with Pool(3) as p:
    p.map(process_input_dir, input_dir_masks)

In [ ]:
# compute deformations in paralel
ifiles = []
ifiles += sorted(glob.glob(f'/data1/{user}/stereoid/RESULTS/SeaIce/SeaIceDrift/pass_original_data*00Z_??_??.npz'))
ifiles += sorted(glob.glob(f'/data1/{user}/stereoid/RESULTS/SeaIce/SeaIceDrift/pass_newer_exp*00Z_??_??.npz'))
ifiles += sorted(glob.glob(f'/data1/{user}/stereoid/RESULTS/SeaIce/SeaIceDrift/pass_sa05free_2019*00Z_??_??.npz'))
with Pool(10) as p:
    p.map(compute_defor, ifiles)

In [ ]:
uv_names = ['uv_i', 'uv_m', 'uv_a']
df_names = ['toti', 'totc']

for ifile in ifiles:
    make_quiclook(ifile, uv_names, df_names)